In [2]:
import psycopg2

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
def get_Redshift_connection():
    host = "grepp-data.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com"
    redshift_user = "chjw102"
    redshift_pass = "Chjw1021!"
    port = 5439
    dbname = "dev"
    conn = psycopg2.connect("dbname={dbname} user={user} host={host} password={password} port={port}".format(
        dbname=dbname,
        user=redshift_user,
        password=redshift_pass,
        host=host,
        port=port
    ))
    conn.set_session(readonly=True, autocommit=True)
    return conn

In [4]:
conn = get_Redshift_connection()

sql_timestamp = "SELECT * FROM raw_data.session_timestamp;"
sql_users = "SELECT * FROM raw_data.user_session_channel;"

In [5]:
import pandas.io.sql as sqlio
import pandas as pd

In [6]:
df_timestamp = sqlio.read_sql_query(sql_timestamp,conn)
df_users = sqlio.read_sql_query(sql_users,conn)

In [7]:
df_timestamp.head()

,sessionid,ts
0,c41dd99a69df04044aa4e33ece9c9249,2019-05-01 00:35:59.897
1,fdc0eb412a84fa549afe68373d9087e9,2019-05-01 02:53:48.503
2,0a54b19a13b6712dc04d1b49215423d8,2019-05-01 12:18:26.637
3,a914ecef9c12ffdb9bede64bb703d877,2019-05-01 13:41:29.267
4,05ae14d7ae387b93370d142d82220f1b,2019-05-01 14:17:54.083


In [8]:
df_users.head()

,userid,sessionid,channel
0,779,7cdace91c487558e27ce54df7cdb299c,Instagram
1,230,94f192dee566b018e0acf31e1f99a2d9,Naver
2,369,7ed2d3454c5eea71148b11d0c25104ff,Youtube
3,248,f1daf122cde863010844459363cd31db,Naver
4,676,fd0efcca272f704a760c3b61dcc70fd0,Instagram


In [9]:
df = pd.merge(df_timestamp, df_users, on='sessionid', how='left')
df.head()

,sessionid,ts,userid,channel
0,c41dd99a69df04044aa4e33ece9c9249,2019-05-01 00:35:59.897,184,Naver
1,fdc0eb412a84fa549afe68373d9087e9,2019-05-01 02:53:48.503,80,Organic
2,0a54b19a13b6712dc04d1b49215423d8,2019-05-01 12:18:26.637,251,Facebook
3,a914ecef9c12ffdb9bede64bb703d877,2019-05-01 13:41:29.267,264,Google
4,05ae14d7ae387b93370d142d82220f1b,2019-05-01 14:17:54.083,744,Facebook


In [10]:
df['year-month'] = df['ts'].apply(lambda x: x.strftime("%Y-%m"))
df.head()

,sessionid,ts,userid,channel,year-month
0,c41dd99a69df04044aa4e33ece9c9249,2019-05-01 00:35:59.897,184,Naver,2019-05
1,fdc0eb412a84fa549afe68373d9087e9,2019-05-01 02:53:48.503,80,Organic,2019-05
2,0a54b19a13b6712dc04d1b49215423d8,2019-05-01 12:18:26.637,251,Facebook,2019-05
3,a914ecef9c12ffdb9bede64bb703d877,2019-05-01 13:41:29.267,264,Google,2019-05
4,05ae14d7ae387b93370d142d82220f1b,2019-05-01 14:17:54.083,744,Facebook,2019-05


In [11]:
grouped = df.groupby('year-month')

In [15]:
grouped.userid.nunique()

year-month
2019-05    281
2019-06    459
2019-07    623
2019-08    662
2019-09    639
2019-10    763
2019-11    721
Name: userid, dtype: int64